## BourbonBaked: Data Enrichment

A new company, BourbonBaked, is launching a new health product offering. Through beta testing they were able to collect user demographic information, but they want as much information as they can get about the income levels of these individuals and their general health so they can create the best lookalike audiences possible for their national product launch next month. As such, they have tasked us with cleaning up their user file and adding in some interesting data to see if we can create some look a like audiences for them to target. 

---

In [2]:
#importing packages needed
import requests
import pandas as pd
import json
import creds

### Accessing User Data File 

Because this is a project about showing the possibilities as opposed to a real company, our user data is generated from an API instead of a file export from a CRM. For this reason, the next few steps go through a little cleanup to get us a usable data set.

In [3]:
#accessing the API to get user data
r = requests.get('https://peoplegeneratorapi.live/api/person/50000')
print(r.status_code)

200


In [4]:
#viewing the data as a dataframe
data=r.json()
df = (pd.DataFrame(data))
df.head()

,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,email,gender,hasDegree,bloodType,username,politicalLeaning,religion,address,doB,gpa
0,Jeremiah Sporer,38,Neuroscientist,85650,794,4956540831566,True,False,166.0,64.4,...,jeremiah.jeremiah@hotmail.com,Male,False,A+,jeremiah6,-0.97,Christianity,"{'streetAddress': '70006 Toy Square', 'city': ...",Wed Jul 23 06:44:07 UTC 1986,3.5
1,Tony Haag,47,Radiologic Technician,60850,515,4976669955491,False,True,156.0,54.7,...,haag@hotmail.com,Male,True,A+,tony87,-0.09,Christianity,"{'streetAddress': '069 Schmidt Land', 'city': ...",Thu Nov 04 06:57:42 UTC 1976,3.2
2,Arlinda Zemlak,40,IT Manager,107427,628,3419-684707-52771,False,True,161.0,59.1,...,arlinda.arlinda@gmail.com,Male,False,A+,arlinda47,-0.39,Hinduism,"{'streetAddress': '797 Bobby Knolls', 'city': ...",Thu May 17 16:35:07 UTC 1984,2.6
3,Sanford Vandervort,52,Social Media Advertising Manager,63844,641,6007-2274-7593-2785,True,False,171.0,55.3,...,vandervort.sanford@outlook.com,Male,False,AB+,sanford65,0.49,Islam,"{'streetAddress': '72264 Abshire Trace', 'city...",Sun Dec 12 04:58:19 UTC 1971,2.5
4,Warren Brakus,6,Student,0,0,None,False,False,166.0,69.5,...,warrenbrakus@yahoo.com,Female,True,A+,warren5,0.11,Islam,"{'streetAddress': '0589 Nellie Curve', 'city':...",Wed Sep 13 11:37:14 UTC 2017,2.5


This is great but it looks like Address is still coming through in a json format, so let's unpack that. 

In [5]:
#normalizing the address data
df = pd.json_normalize(data)

#getting a list of all the column names
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   50000 non-null  object 
 1   age                    50000 non-null  int64  
 2   job                    50000 non-null  object 
 3   incomeUSD              50000 non-null  int64  
 4   creditScore            50000 non-null  int64  
 5   ccNumber               35707 non-null  object 
 6   married                50000 non-null  bool   
 7   hasChildren            50000 non-null  bool   
 8   height                 50000 non-null  float64
 9   weight                 50000 non-null  float64
 10  eyeColor               50000 non-null  object 
 11  email                  50000 non-null  object 
 12  gender                 50000 non-null  object 
 13  hasDegree              50000 non-null  bool   
 14  bloodType              50000 non-null  object 
 15  us

Since we know we are only going to market this product to adults, we need to remove all children from the dataset 

In [6]:
#removing individuals under the age of 18 
df.drop(df[df['age'] < 18].index, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35707 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   35707 non-null  object 
 1   age                    35707 non-null  int64  
 2   job                    35707 non-null  object 
 3   incomeUSD              35707 non-null  int64  
 4   creditScore            35707 non-null  int64  
 5   ccNumber               35707 non-null  object 
 6   married                35707 non-null  bool   
 7   hasChildren            35707 non-null  bool   
 8   height                 35707 non-null  float64
 9   weight                 35707 non-null  float64
 10  eyeColor               35707 non-null  object 
 11  email                  35707 non-null  object 
 12  gender                 35707 non-null  object 
 13  hasDegree              35707 non-null  bool   
 14  bloodType              35707 non-null  object 
 15  us

As we can see the file is now smaller and excludes anyone under the age of 18. 

Additionally, we only want a file of United States residents since our mock company is US based and only marketing to those areas.  

In [7]:
#removing non USA based users
df = df[df['address.country'] == 'United States']

df.head()

,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,gpa,address.streetAddress,address.city,address.state,address.country,address.zip,address.geonameId,address.phoneNumber,address.ipAddress,address.countryCode
85,Charolette Carroll,42,Anthropologist,71247,823,6007-2224-9881-3450,False,False,166.0,80.8,...,3.3,110 Harris Neck,Schaumburg,North Dakota,United States,41882,1185128,+880 7639450439,50.113.57.240,BD
107,Lizette Muller,45,Uber Driver,31092,639,6771-8987-0818-5229,True,True,171.0,65.5,...,2.8,9400 Ratke Heights,Summerville,Arizona,United States,99853-5960,4161797,+1 42487290427,67.24.48.13,US
225,Ezra Champlin,19,Nuclear Medicine Technologist,82922,558,3742-496036-15098,False,False,160.0,55.0,...,1.6,244 Balistreri Union,East Meadow,Connecticut,United States,25794,1820187,+673 9114799,177.39.38.252,BN
247,Luana Jacobi,23,Retired,16786,797,6767-8698-8214-3742-85,True,False,160.0,49.6,...,2.5,33052 Ramon Drive,Artesia,New York,United States,16572-6103,912764,+260 758696783,219.35.20.4,ZM
379,Wilmer Skiles,33,Psychiatrist,122929,543,6709009668675160,True,True,175.0,66.0,...,2.8,9887 Satterfield Stravenue,Westminster,Washington,United States,15001,203104,+250 026787320,75.54.223.158,RW


Great! Now that we have a complete intial user file, time to enrich it

---

### Enriching the Data

We know BourbonBaked is interested in the vital statistics of their users since this is a weight loss product. They are interested in knowing if their beta users fell into any significant category. Since we have height and weight already, we can use this information to get their BMI. 

The BMI API we are using requires height in inches and weight in pounds, so we will first need to update this in our dataframe as well before hitting the API. Currently, the Height is in CM and the Weight is in KG. 

- **Height**: Convert from CM to In, we need to divide the Height column values by 2.54 in order to get the correct value in inches
- **Weight**: Convert from KG to LB, we need to multiply the Weight column values by 2.205 in order to get the correct value in pounds

In [8]:
#adding height in inches and weight in pounds
df['height_inches'] = df['height'] / 2.54

df['weight_pounds'] = df['weight'] * 2.205

In [9]:
#generating just the columns needed to double check
bmi_columns_needed = df[['age','height','weight','height_inches','weight_pounds']]
print(bmi_columns_needed)

       age  height  weight  height_inches  weight_pounds
85      42   166.0    80.8      65.354331       178.1640
107     45   171.0    65.5      67.322835       144.4275
225     19   160.0    55.0      62.992126       121.2750
247     23   160.0    49.6      62.992126       109.3680
379     33   175.0    66.0      68.897638       145.5300
...    ...     ...     ...            ...            ...
49226   22   160.0    59.2      62.992126       130.5360
49420   30   161.0    70.1      63.385827       154.5705
49436   30   161.0    74.8      63.385827       164.9340
49694   28   151.0    80.1      59.448819       176.6205
49960   52   166.0    59.2      65.354331       130.5360

[296 rows x 5 columns]


#### Appending in BMI Data 

In [10]:
#BMI function for API
def calculate_bmi(row):
    url = "https://fitness-calculator.p.rapidapi.com/bmi"
    querystring = {
        "age": row['age'],
        "weight": row['weight'],
        "height": row['height']
    }
    headers = {
        "X-RapidAPI-Key": creds.api_key,
        "X-RapidAPI-Host": "fitness-calculator.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    
    # Check if 'data' key is present in the API response
    if 'data' in data:
        bmi = data['data'].get('bmi', None)
        health = data['data'].get('health', None)
        healthy_bmi_range = data['data'].get('healthy_bmi_range', None)
        return bmi, health, healthy_bmi_range
    else:
        return None, None, None  

#add new columns to the DataFrame
df['bmi'], df['health'], df['healthy_bmi_range'] = zip(*df.apply(calculate_bmi, axis=1))

# Display the updated DataFrame
df.head()

,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,address.zip,address.geonameId,address.phoneNumber,address.ipAddress,address.countryCode,height_inches,weight_pounds,bmi,health,healthy_bmi_range
85,Charolette Carroll,42,Anthropologist,71247,823,6007-2224-9881-3450,False,False,166.0,80.8,...,41882,1185128,+880 7639450439,50.113.57.240,BD,65.354331,178.1640,29.32,Overweight,18.5 - 25
107,Lizette Muller,45,Uber Driver,31092,639,6771-8987-0818-5229,True,True,171.0,65.5,...,99853-5960,4161797,+1 42487290427,67.24.48.13,US,67.322835,144.4275,22.40,Normal,18.5 - 25
225,Ezra Champlin,19,Nuclear Medicine Technologist,82922,558,3742-496036-15098,False,False,160.0,55.0,...,25794,1820187,+673 9114799,177.39.38.252,BN,62.992126,121.2750,21.48,Healthy weight,5 - 85
247,Luana Jacobi,23,Retired,16786,797,6767-8698-8214-3742-85,True,False,160.0,49.6,...,16572-6103,912764,+260 758696783,219.35.20.4,ZM,62.992126,109.3680,19.37,Normal,18.5 - 25
379,Wilmer Skiles,33,Psychiatrist,122929,543,6709009668675160,True,True,175.0,66.0,...,15001,203104,+250 026787320,75.54.223.158,RW,68.897638,145.5300,21.55,Normal,18.5 - 25


Now that we have added in BMI, Health indicator, and the healthy BMI range, I would like to make the data set more robust to make insights easier. BourbonBaked also mentioned wanting to know income levels to see if their hitting their anticipated target demographic. 

First, I will take the income column and add a new range that signifies lower, middle, or upper class. 

In [11]:
def categorize_income_level(incomeUSD):
    income = int(incomeUSD) 
    
    if income < 52200:
        return 'Low income'
    elif 52200 <= income < 156600:
        return 'Middle income'
    else:
        return 'Upper income'

# Add a new column 'income level' to the DataFrame with the income level categories
df['income level'] = df['incomeUSD'].apply(categorize_income_level)

df.head()


,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,address.geonameId,address.phoneNumber,address.ipAddress,address.countryCode,height_inches,weight_pounds,bmi,health,healthy_bmi_range,income level
85,Charolette Carroll,42,Anthropologist,71247,823,6007-2224-9881-3450,False,False,166.0,80.8,...,1185128,+880 7639450439,50.113.57.240,BD,65.354331,178.1640,29.32,Overweight,18.5 - 25,Middle income
107,Lizette Muller,45,Uber Driver,31092,639,6771-8987-0818-5229,True,True,171.0,65.5,...,4161797,+1 42487290427,67.24.48.13,US,67.322835,144.4275,22.40,Normal,18.5 - 25,Low income
225,Ezra Champlin,19,Nuclear Medicine Technologist,82922,558,3742-496036-15098,False,False,160.0,55.0,...,1820187,+673 9114799,177.39.38.252,BN,62.992126,121.2750,21.48,Healthy weight,5 - 85,Middle income
247,Luana Jacobi,23,Retired,16786,797,6767-8698-8214-3742-85,True,False,160.0,49.6,...,912764,+260 758696783,219.35.20.4,ZM,62.992126,109.3680,19.37,Normal,18.5 - 25,Low income
379,Wilmer Skiles,33,Psychiatrist,122929,543,6709009668675160,True,True,175.0,66.0,...,203104,+250 026787320,75.54.223.158,RW,68.897638,145.5300,21.55,Normal,18.5 - 25,Middle income


Next lets add an indicator for where in the United States an individual lives. 

In [12]:
regions = {
    "Northeast": ["Connecticut", "Maine","Massachusetts", "New Hampshire", "Rhode Island", "Vermont", "New Jersey", "New York", "Pennsylvania"], 
    "Midwest": ["Indiana", "Illinois", "Michigan", "Ohio", "Wisconsin", "Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska", "North Dakota", "South Dakota"],
    "South" : ["Deleware", "District of Columbia", "Florida", "Georgia", "Maryland", "North Carolina", "South Carolina", "Virginia", "West Virginia", "Alabama", "Kentucky", "Mississippi", "Tennessee", "Arkansas", "Louisiana", "Oklahoma", "Texas"],
    "West" : ["Arizona", "Colorada", "Idaho", "New Mexico", "Montana", "Utah", "Nevada", "Wyoming", "Alaska", "California", "Hawaii", "Oregon", "Washington"]}

# Function to map states to regions
def map_state_to_region(state):
    for region, states_list in regions.items():
        if state in states_list:
            return region
    return 'Unknown' 

# Add the "Regions" column to the DataFrame using the map function
df['region'] = df['address.state'].map(map_state_to_region)

df.head()

,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,address.phoneNumber,address.ipAddress,address.countryCode,height_inches,weight_pounds,bmi,health,healthy_bmi_range,income level,region
85,Charolette Carroll,42,Anthropologist,71247,823,6007-2224-9881-3450,False,False,166.0,80.8,...,+880 7639450439,50.113.57.240,BD,65.354331,178.1640,29.32,Overweight,18.5 - 25,Middle income,Midwest
107,Lizette Muller,45,Uber Driver,31092,639,6771-8987-0818-5229,True,True,171.0,65.5,...,+1 42487290427,67.24.48.13,US,67.322835,144.4275,22.40,Normal,18.5 - 25,Low income,West
225,Ezra Champlin,19,Nuclear Medicine Technologist,82922,558,3742-496036-15098,False,False,160.0,55.0,...,+673 9114799,177.39.38.252,BN,62.992126,121.2750,21.48,Healthy weight,5 - 85,Middle income,Northeast
247,Luana Jacobi,23,Retired,16786,797,6767-8698-8214-3742-85,True,False,160.0,49.6,...,+260 758696783,219.35.20.4,ZM,62.992126,109.3680,19.37,Normal,18.5 - 25,Low income,Northeast
379,Wilmer Skiles,33,Psychiatrist,122929,543,6709009668675160,True,True,175.0,66.0,...,+250 026787320,75.54.223.158,RW,68.897638,145.5300,21.55,Normal,18.5 - 25,Middle income,West


We'd also like to add in some state specific information for our analysis, so we'll use another API to bring in data for all states

In [13]:
import requests

url2 = "https://us-states.p.rapidapi.com/basic"

headers = {
	"X-RapidAPI-Key": creds.api_key,
	"X-RapidAPI-Host": "us-states.p.rapidapi.com"
}

response2 = requests.get(url2, headers=headers)

print(response2.json())


[{'name': 'Alabama', 'postal': 'AL', 'capital': {'name': 'Montgomery', 'latitude': '32.377716', 'longitude': '-86.300568'}, 'population': {'density_km': '37', 'total': '5024279', 'density_mi': '95'}}, {'name': 'Alaska', 'postal': 'AK', 'capital': {'name': 'Juneau', 'latitude': '58.301598', 'longitude': '-134.420212'}, 'population': {'density_km': '<1', 'total': '733391', 'density_mi': '1'}}, {'name': 'American Samoa', 'postal': 'AS', 'capital': {'name': 'Pago Pago', 'latitude': '-14.279444', 'longitude': '-170.700556'}, 'population': {'density_km': '279', 'total': '49710', 'density_mi': '721'}}, {'name': 'Arizona', 'postal': 'AZ', 'capital': {'name': 'Phoenix', 'latitude': '33.448143', 'longitude': '-112.096962'}, 'population': {'density_km': '23', 'total': '7151502', 'density_mi': '60'}}, {'name': 'Arkansas', 'postal': 'AR', 'capital': {'name': 'Little Rock', 'latitude': '34.746613', 'longitude': '-92.288986'}, 'population': {'density_km': '22', 'total': '3011524', 'density_mi': '57'}

In [14]:
#viewing the data as a dataframe
data2=response2.json()
df_state = (pd.DataFrame(data2))
df_state.head()

,name,postal,capital,population
0,Alabama,AL,"{'name': 'Montgomery', 'latitude': '32.377716'...","{'density_km': '37', 'total': '5024279', 'dens..."
1,Alaska,AK,"{'name': 'Juneau', 'latitude': '58.301598', 'l...","{'density_km': '<1', 'total': '733391', 'densi..."
2,American Samoa,AS,"{'name': 'Pago Pago', 'latitude': '-14.279444'...","{'density_km': '279', 'total': '49710', 'densi..."
3,Arizona,AZ,"{'name': 'Phoenix', 'latitude': '33.448143', '...","{'density_km': '23', 'total': '7151502', 'dens..."
4,Arkansas,AR,"{'name': 'Little Rock', 'latitude': '34.746613...","{'density_km': '22', 'total': '3011524', 'dens..."


In [15]:
df_state = pd.json_normalize(data2)

df_state.head()

,name,postal,capital.name,capital.latitude,capital.longitude,population.density_km,population.total,population.density_mi
0,Alabama,AL,Montgomery,32.377716,-86.300568,37,5024279,95
1,Alaska,AK,Juneau,58.301598,-134.420212,<1,733391,1
2,American Samoa,AS,Pago Pago,-14.279444,-170.700556,279,49710,721
3,Arizona,AZ,Phoenix,33.448143,-112.096962,23,7151502,60
4,Arkansas,AR,Little Rock,34.746613,-92.288986,22,3011524,57


In [16]:
df_state.rename(columns={'name': 'state'}, inplace=True)

df_state.head()

,state,postal,capital.name,capital.latitude,capital.longitude,population.density_km,population.total,population.density_mi
0,Alabama,AL,Montgomery,32.377716,-86.300568,37,5024279,95
1,Alaska,AK,Juneau,58.301598,-134.420212,<1,733391,1
2,American Samoa,AS,Pago Pago,-14.279444,-170.700556,279,49710,721
3,Arizona,AZ,Phoenix,33.448143,-112.096962,23,7151502,60
4,Arkansas,AR,Little Rock,34.746613,-92.288986,22,3011524,57


I'd like to add population total into the existing dataset we have so we can compare the state population to extrapolate potential new user data

In [17]:
#perform merge based on the state names
merged_df = df.merge(df_state[['state', 'population.total']], left_on='address.state', right_on='state', how='left')

#rename the 'population.total' column 
merged_df.rename(columns={'population.total': 'state_population'}, inplace=True)

print(merged_df)

                   name  age                                      job  \
0    Charolette Carroll   42                           Anthropologist   
1        Lizette Muller   45                              Uber Driver   
2         Ezra Champlin   19            Nuclear Medicine Technologist   
3          Luana Jacobi   23                                  Retired   
4         Wilmer Skiles   33                             Psychiatrist   
..                  ...  ...                                      ...   
291     Randall Quigley   22  Environmental Health and Safety Manager   
292       Dominic Braun   30                         Machine Operator   
293       Rickey Kohler   30                          Cinematographer   
294    Fernande Pacocha   28                        Landfill Operator   
295        Ismael Wolff   52                            Event Planner   

     incomeUSD  creditScore                 ccNumber  married  hasChildren  \
0        71247          823      6007-2224-98

In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 295
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   name                   296 non-null    object 
 1   age                    296 non-null    int64  
 2   job                    296 non-null    object 
 3   incomeUSD              296 non-null    int64  
 4   creditScore            296 non-null    int64  
 5   ccNumber               296 non-null    object 
 6   married                296 non-null    bool   
 7   hasChildren            296 non-null    bool   
 8   height                 296 non-null    float64
 9   weight                 296 non-null    float64
 10  eyeColor               296 non-null    object 
 11  email                  296 non-null    object 
 12  gender                 296 non-null    object 
 13  hasDegree              296 non-null    bool   
 14  bloodType              296 non-null    object 
 15  userna

There is one last data point to add in, and that is the Per Capita Personal Income for each state to append into the file. This allows us to see how a user compares within their state for state segmented marketing (if needed). 

In [19]:
df_state_income = pd.read_csv('data_file\states.csv')
df_state_income.head()

,Rank,State,FIPS Code,PCPI ($)
0,1,District of Columbia,11000,"96,728"
1,2,Connecticut,9000,"84,972"
2,3,Massachusetts,25000,"84,945"
3,4,New Jersey,34000,"78,700"
4,5,New York,36000,"78,089"


In [20]:
#adding in PCPI (Per Capita Personal Income) to the merged dataframe

merged_df = merged_df.merge(df_state_income, left_on='state', right_on='State', how='left')

merged_df.rename(columns={'PCPI ($)': 'state_per_capita_income'}, inplace=True)

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 295
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     296 non-null    object 
 1   age                      296 non-null    int64  
 2   job                      296 non-null    object 
 3   incomeUSD                296 non-null    int64  
 4   creditScore              296 non-null    int64  
 5   ccNumber                 296 non-null    object 
 6   married                  296 non-null    bool   
 7   hasChildren              296 non-null    bool   
 8   height                   296 non-null    float64
 9   weight                   296 non-null    float64
 10  eyeColor                 296 non-null    object 
 11  email                    296 non-null    object 
 12  gender                   296 non-null    object 
 13  hasDegree                296 non-null    bool   
 14  bloodType                2

In [21]:
#cleaning up the final dataframe 

merged_df.drop(columns = ['Rank', 'FIPS Code', 'State', 'state'],axis=1, inplace=True)

print(merged_df.columns)

Index(['name', 'age', 'job', 'incomeUSD', 'creditScore', 'ccNumber', 'married',
       'hasChildren', 'height', 'weight', 'eyeColor', 'email', 'gender',
       'hasDegree', 'bloodType', 'username', 'politicalLeaning', 'religion',
       'doB', 'gpa', 'address.streetAddress', 'address.city', 'address.state',
       'address.country', 'address.zip', 'address.geonameId',
       'address.phoneNumber', 'address.ipAddress', 'address.countryCode',
       'height_inches', 'weight_pounds', 'bmi', 'health', 'healthy_bmi_range',
       'income level', 'region', 'state_population',
       'state_per_capita_income'],
      dtype='object')


In [22]:
#exporting final dataframe as csv for further analysis 

merged_df.to_csv('enriched_customer_file.csv')

That's a wrap on data wrangling! The cleaned final file is now available in a csv export. In *data_vizualization.ipynb* we'll explore the data set as a whole and break it into segments to see what insights we can garner. 